In [1]:
import pandas as pd
import numpy as np

In [2]:
df1 = pd.read_csv('Data/1_monday.csv',sep=';')
# df1['monday']='monday'
df1['new_customer_no'] = df1['customer_no'].astype(str) +'_'+ 'monday'

df2 = pd.read_csv('Data/2_tuesday.csv',sep=';')
# df2['tuesday']='tuesday'
df2['new_customer_no'] = df2['customer_no'].astype(str) +'_'+ 'tuesday'

df3 = pd.read_csv('Data/3_wednesday.csv',sep=';')
# df3['wednesday']='wednesday'
df3['new_customer_no'] = df3['customer_no'].astype(str) +'_'+ 'wednesday'

df4 = pd.read_csv('Data/4_thursday.csv',sep=';')
# df4['thursday']='thursday'
df4['new_customer_no'] = df4['customer_no'].astype(str) +'_'+ 'thursday'

df5 = pd.read_csv('Data/5_friday.csv',sep=';')
# df5['friday']='friday'
df5['new_customer_no'] = df5['customer_no'].astype(str) +'_'+ 'friday'

df = pd.concat([df1,df2,df3,df4,df5], axis=0)
df = df.drop('customer_no',axis=1)
df = df[['timestamp','new_customer_no','location']]
df.rename(columns={'new_customer_no': 'customer_no'}, inplace=True)
df100 = df.copy()
df.head(5)

,timestamp,customer_no,location
0,2019-09-02 07:03:00,1_monday,dairy
1,2019-09-02 07:03:00,2_monday,dairy
2,2019-09-02 07:04:00,3_monday,dairy
3,2019-09-02 07:04:00,4_monday,dairy
4,2019-09-02 07:04:00,5_monday,spices


In [3]:
df['timestamp'] = pd.to_datetime(df['timestamp'])

In [4]:
df101 =df100.copy()
df101['timestamp'] = pd.to_datetime(df101['timestamp'])
df101 = df101.set_index('timestamp')
df101 = df101.groupby(['customer_no'])[['location']].resample(rule="Min").asfreq().ffill()
df101

location
customer_no timestamp                    
1000_friday 2019-09-06 17:19:00     fruit
            2019-09-06 17:20:00  checkout
1000_monday 2019-09-02 17:44:00     dairy
            2019-09-02 17:45:00     dairy
            2019-09-02 17:46:00     dairy
...                                   ...
9_tuesday   2019-09-03 07:28:00     dairy
            2019-09-03 07:29:00     dairy
            2019-09-03 07:30:00  checkout
9_wednesday 2019-09-04 07:03:00    spices
            2019-09-04 07:04:00  checkout

[53903 rows x 1 columns]

In [5]:
# g['before'] = g['places'].shift(1)
df101['before'] = df101['location'].shift(1)
df101.head(5)

location    before
customer_no timestamp                              
1000_friday 2019-09-06 17:19:00     fruit       NaN
            2019-09-06 17:20:00  checkout     fruit
1000_monday 2019-09-02 17:44:00     dairy  checkout
            2019-09-02 17:45:00     dairy     dairy
            2019-09-02 17:46:00     dairy     dairy

In [6]:
# df = df[df.line_race != 0]
# df101 = df101[df101.before != 'checkout']
df102 = df101 


In [7]:
df101['dummy'] = True

In [8]:
df101.dropna().groupby(by=['before','location'])[['dummy']].count().unstack()

dummy                                 
location checkout    dairy  drinks   fruit  spices
before                                            
checkout      NaN   2136.0  1137.0  2798.0  1345.0
dairy      1825.0  13098.0  1044.0   891.0   916.0
drinks     2096.0    106.0  5822.0   856.0   846.0
fruit      2555.0   1221.0   699.0  7587.0   646.0
spices      941.0   1213.0  1024.0   575.0  2525.0

In [9]:
pd.crosstab( df102.before, df102.location, normalize = 1)

location,checkout,dairy,drinks,fruit,spices
before,,,,,
checkout,0.000000,0.120176,0.116903,0.220194,0.214240
dairy,0.246056,0.736919,0.107341,0.070119,0.145906
drinks,0.282594,0.005964,0.598602,0.067364,0.134756
fruit,0.344479,0.068696,0.071869,0.597072,0.102899
spices,0.126871,0.068246,0.105285,0.045251,0.402198
